<a href="https://colab.research.google.com/github/ingoscargiraldor/ingoscargiraldor/blob/main/data-quality-tutorial.ipynb#scrollTo=xDdidgd_dSDV" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))
  
Last update: September, 2023

In [1]:
!pip install pylev

In [2]:
import re
from random import randint
from datetime import datetime
from difflib import SequenceMatcher

import numpy as np
import pandas as pd

import pylev

In [3]:
pd.set_option("display.max_columns", None)

## 1. Loading the data

In [4]:
homicides_df = pd.read_csv("./data/homicides.csv")

In [5]:
homicides_df.shape

(12400, 22)

In [6]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [7]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
6662,07/15/2021 12:00:00 AM,RISARALDA,SANTA ROSA DE CABAL,Miwrcoles,23:35,B. LA ESTACION,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,30.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,66682000.0,1985.0,50-053,mkgosu5685@unidatos.edu.co
2321,03/12/2021 12:00:00 AM,VALLE,CALI (CT),Juees,0:15,ANTONIO NARIÑO E16,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,30.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1985.0,72-217,tlhbsr7947@unidatos.edu.co
9200,10/02/2021 12:00:00 AM,META,VILLAVICENCIO (CT),Viernes,3:15,SANTA FE,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,35.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,NO REPORTADO,50001000.0,1980.0,68-851,pmjocd9755@gmail.com
209,01/04/2021 12:00:00 AM,MAGDALENA,FUNDACIÓN,Domingo,16:50,LA ESPERANZA (CNO REPORTADO4),URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,52.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,47288000.0,1963.0,83-611,sjlabb8052@unidatos.edu.co
12186,12/26/2021 12:00:00 AM,CHOCÓ,ISTMINA,Sábado,6:20,SANFRANCISCO,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,27361000.0,1988.0,59-681,mlmorh6835@unidatos.edu.co


## 2. Working with datetimes

In [8]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [9]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

ValueError: ignored

In [10]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [11]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
486,13/12/2021 12:00:00 AM,VALLE,CALI (CT),kunes,23:00,MOJICA E15,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,89.0,80-330,lujhdf9132@gmail.com
695,30/01/2021 12:00:00 AM,BOLÍVAR,CARTAGENA (CT),Lunes,5:30,REP. DEL LIBANO,URBANA,VIAS PUBLICAS,ARMA BLANCA,NO REPORTADO,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,13001000.0,1990.0,12-915,ghumtg4094@unidatos.edu.co
1250,18/05/2021 12:00:00 AM,HUILA,TESALIA,Jueves,19:30,VEREDA PACARNI,RURAL,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,34.0,FEMENINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41797000.0,1981.0,99-095,sdaggf6639@gmail.com
12168,12/25/2021 12:00:00 MM,VALLE,PALMIRA,Viernes,15:30,LA EMILIA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,17.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1998.0,16-362,uschca1775@gmail.com
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [13]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [14]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
HORA                         object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [16]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12395
Name: FECHA, dtype: int64

*The homicide hour is available in a different column!*

In [17]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11])+homicides_df["HORA"]

In [18]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

In [19]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [20]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [21]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [22]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
1494,2021-02-14 20:25:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Sábado,URB. LA CORUÑA ENO REPORTADO19,URBANA,CANCHA DE FUTBOL,ARMA DE FUEGO,A PIE,A PIE,33.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1982.0,95-715,aecifb2847@unidatos.edu.co
7932,2021-08-24 21:12:00,CUNDINAMARCA,EL COLEGIO,Lunes,VER.EL PORVENIR,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,39.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,25245000.0,1976.0,99-707,cnqute3163@gmail.com
10676,2021-11-16 16:40:00,VALLE,CALI (CT),Lnues,LOS COMUNEROS II E13,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,17.0,MASCULINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1998.0,84-178,eukmpe3913@unidatos.edu.co
312,2021-01-08 10:15:00,NARIÑO,RICAURTE,Jueves,OSPINA PEREZ,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,39.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,52612000.0,1976.0,24-095,bdgkoq1981@unidatos.edu.co
5963,2021-06-26 23:30:00,ANTIOQUIA,MEDELLÍN (CT),Viernes,EL PINAL CNO REPORTADO8,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,32.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1983.0,76-287,qubnrs3063@unidatos.edu.co


## 2. Fixing categorical column

In [23]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [24]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

In [25]:
homicides_df = homicides_df.replace('SAN ANDRÉS', 'SAN ANDRÉS Y PROVIDENCIA')

In [26]:
homicides_df = homicides_df.replace('N. DE SANTANDER', 'NORTE DE SANTANDER')

In [27]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

## 3. Analyzing potential duplicates

In [28]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [29]:
duplicates_by_cedula.shape

(1627, 21)

In [30]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [31]:
clear_df = homicides_df.drop_duplicates(subset=['CÉDULA'])
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula_2 = clear_df.loc[clear_df["CÉDULA"].duplicated(keep=False)]
duplicates_by_cedula_2.sort_values("CÉDULA", ascending=True).head(6)


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO


## 4. Fixing formats

In [32]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [33]:
cedula_malformed.shape

(28, 21)

In [34]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [35]:
len(cedula_malformed)

28

In [36]:
cedula_malformed_2 = homicides_df
cedula_malformed_2["CÉDULA"] = homicides_df["CÉDULA"].str.replace('-', '')
cedula_malformed_2["CÉDULA"] = cedula_malformed_2["CÉDULA"].str.replace(r'^(\d{2})(\d{3})$', r'\1-\2')
cedula_malformed_2.head(6)

<ipython-input-36-a8868aa26215>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  cedula_malformed_2["CÉDULA"] = cedula_malformed_2["CÉDULA"].str.replace(r'^(\d{2})(\d{3})$', r'\1-\2')


,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
5,2021-01-01 00:47:00,ANTIOQUIA,MEDELLÍN (CT),Juees,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1973.0,54-450,grlcsj8696@unidatos.edu.co


In [37]:
cedula_malformed = cedula_malformed_2.loc[cedula_malformed_2["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]
len(cedula_malformed)

0

In [38]:
len(homicides_df)

12395

In [39]:
len(cedula_malformed_2)

12395

In [40]:
cedula_malformed_2.iloc[172]

FECHA                 2021-01-03 11:00:00
DEPARTAMENTO                         META
MUNICIPIO                         MESETAS
DIA                                Sábado
BARRIO                    VEREDA EL CAFRE
ZONA                                RURAL
CLASE DE SITIO             ZONA SELVÁTICA
ARMA O MEDIO             MINA ANTIPERSONA
MOVIL VICTIMA                       A PIE
MOVIL AGRESOR                       A PIE
EDAD                                 24.0
GENERO                          MASCULINO
ESTADO CIVIL                      SOLTERO
CLASE EMPLEADO          EMPLEADO EJERCITO
PROFESION                    NO REPORTADO
ESCOLARIDAD                    SECUNDARIA
PAIS NACE                        COLOMBIA
CODIGO DANE                    50330000.0
AÑO DE NACIMIENTO                  1991.0
CÉDULA                             68-021
CORREO               oohghd8899@gmail.com
Name: 172, dtype: object

<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [48]:
mail_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match("^[^\W\d_]*\d{4}@(?:[a-zA-Z0-9-]+\.)+(?:edu\.co|com)$", x) is None))]
mail_malformed.shape

(2293, 21)

In [49]:
mail_malformed.head(10)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
28,2021-01-01 15:00:00,CAQUETÁ,SAN JOSÉ DEL FRAGUA,Jueces,CENTRO,URBANA,DISCOTECAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18610000.0,1991.0,94-565,correo975@unidatos.edu.co
36,2021-01-01 08:00:00,CAUCA,PATÍA,Jueves,C/MIENTO PIEDRASENTADA,RURAL,CARRETERA,ARMA DE FUEGO,VEHICULO,A PIE,26.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19532000.0,1989.0,69-084,correo4714@colombia.gov.co
47,2021-01-01 09:00:00,CHOCÓ,TADÓ,Jueves,POPULAR,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,34.0,MASCULINO,CASADO,EMPLEADO POLICIAL,POLICIA,TECNICO,COLOMBIA,27787000.0,1981.0,29-957,correo8297@colombia.gov.co
55,2021-01-01 01:30:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,ÁLVARO BERNAL SEGURA E-19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1990.0,35-260,correo7285@colombia.gov.co
60,2021-01-01 09:50:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,LAS COLINAS E-18,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,37.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1978.0,96-322,correo870@unidatos.edu.co
63,2021-01-01 17:53:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueces,LA ESTRELLA SECTOR LAGOS E-19,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,19.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1996.0,45-426,correo102@unidatos.edu.co
79,2021-01-01 16:00:00,NARIÑO,GUAITARILLA,Jueves,VEREDA AHUMADA CHIQUITA,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,26.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,52320000.0,1989.0,54-950,correo912@unidatos.edu.co
84,2021-01-01 08:40:00,NARIÑO,SAN ANDRES DE TUMACO,Jueces,PANAMA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,52835000.0,1989.0,55-611,correo8342@colombia.gov.co
85,2021-01-01 08:11:00,PUTUMAYO,VALLE DEL GUAMUEZ,Jueves,EL EDEN,URBANA,TERMINAL DE TRANSPORTES,ARMA BLANCA,A PIE,A PIE,28.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,86865000.0,1987.0,37-347,correo670@unidatos.edu.co
95,2021-01-01 04:34:00,TOLIMA,GUAMO,Juees,INVASION LA CARRILERA,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,21.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,73319000.0,1994.0,73-094,correo863@unidatos.edu.co


## 5. Automating imputation of categorical values

In [50]:
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [51]:
homicides_df["DIA"].value_counts()

Domingo      2069
Sábado       1400
Lunes        1295
Jueves       1248
Viernes      1184
Martes       1113
Miércoles    1053
Juees         162
Domungo       151
domnigo       146
domungo       137
Dominog       133
Doningo       129
Sávado        102
Sabado        100
Viermes        99
Voernes        96
lune           89
virnes         87
Vierens        85
iernes         84
sábad          84
Mates          83
Jueces         83
Ssbado         82
juves          82
Lnues          81
Luns           80
Sabadi         80
Miwrcoles      78
kunes          70
miércles       69
Msrtes         66
Mircoles       65
Maryes         65
mates          64
Lumes          64
Juevrs         62
Marte          62
Miercoles      58
Mirrcoles      55
Name: DIA, dtype: int64

In [52]:
# Calculating the distance between two words using the Levenshtein distance
pylev.levenshtein("sábado", "sabaod")

3

In [53]:
pylev.levenshtein("sábado", "viernes")

7

In [54]:
SequenceMatcher(None, "sábado", "sabaod").ratio()

0.6666666666666666

In [55]:
SequenceMatcher(None, "sábado", "viernes").ratio()

0.15384615384615385

<span style="color:red">How does SequenceMatcher works? How this differ from the Levenshtein distance?</span>

 SequenceMatcher implementa un método heurístico que identifica automáticamente a ciertos elementos como no deseados. El método heurístico consiste en contar cuantas veces aparece cada elemento en la secuencia. Si las apariciones del duplicado de un elemento (después del primero) contabilizan mas del 1% de la secuencia

 SequenceMatcher se utiliza cuando se requiere una medida mas precisa y Levenshtein cuando se requiere de una medida rapida

<span style="color:red">TODO: Create a function to fix the digitation errors for the "DIA" column.</span>

In [56]:
days_list = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]


In [57]:
days_list = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
days_aut = homicides_df
def encontrar_dia_similar(valor):
  mejor_coincidencia = None
  mejor_distancia = float('inf')
  for dia in days_list:
      distancia = pylev.levenshtein(valor.lower(), dia.lower())
      if distancia < mejor_distancia:
          mejor_coincidencia = dia
          mejor_distancia = distancia
  return mejor_coincidencia
days_aut['DIA'] = days_aut['DIA'].apply(encontrar_dia_similar)
days_aut["DIA"].unique()

array(['Jueves', 'Viernes', 'Sábado', 'Domingo', 'Lunes', 'Martes',
       'Miércoles'], dtype=object)

In [58]:
days_aut.head(100)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Jueves,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-01-01 04:34:00,TOLIMA,GUAMO,Jueves,INVASION LA CARRILERA,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,21.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,73319000.0,1994.0,73-094,correo863@unidatos.edu.co
96,2021-01-01 00:00:00,TOLIMA,IBAGUÉ (CT),Jueves,COMUNEROS,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,34.0,MASCULINO,CASADO,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,73001000.0,1981.0,17-670,cgtpjq5472@gmail.com
97,2021-01-01 03:01:00,TOLIMA,MARIQUITA,Jueves,EL DORADO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,23.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,73443000.0,1992.0,26-683,lfkdpc3495@unidatos.edu.co
98,2021-01-01 09:00:00,TOLIMA,ORTEGA,Jueves,VRDA. LOS GUAYABOS,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,70.0,MASCULINO,SEPARADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,73504000.0,1945.0,95-410,umcrgu3348@unidatos.edu.co


In [59]:
days_aut[days_aut['DIA'] == 'Miércoles']

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
274,2021-01-07 18:15:00,ANTIOQUIA,BARBOSA,Miércoles,BARRIO DE JESUS,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,5079000.0,86.0,62-083,ocolgu9647@gmail.com
275,2021-01-07 17:30:00,ANTIOQUIA,CISNEROS,Miércoles,CAMELIAS,RURAL,FINCAS Y SIMILARES,ARMA DE FUEGO,A PIE,A PIE,20.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5190000.0,1995.0,34-931,oqlifs1273@gmail.com
276,2021-01-07 00:00:00,ARAUCA,SARAVENA,Miércoles,VEREDA PUERTO NARIÑO,RURAL,VIAS PUBLICAS,CONTUNDENTES,A PIE,A PIE,70.0,MASCULINO,SOLTERO,COMERCIANTE,NO REPORTADO,PRIMARIA,COLOMBIA,81736000.0,1945.0,55-128,uaupkg5351@unidatos.edu.co
277,2021-01-07 21:00:00,ATLÁNTICO,BARRANQUILLA (CT),Miércoles,CARRIZAL,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,PASAJERO MOTOCICLETA,43.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,TECNICO,COLOMBIA,8001000.0,1972.0,72-066,correo540@unidatos.edu.co
278,2021-01-07 18:00:00,ATLÁNTICO,SABANALARGA,Miércoles,PRIMERO DE MAYO,URBANA,DENTRO DE LA VIVIENDA,CONTUNDENTES,A PIE,A PIE,1.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,ANALFABETA,COLOMBIA,8638000.0,2014.0,35-163,qjlump7011@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12343,2021-12-30 22:20:00,VALLE,CANDELARIA,Miércoles,PANAMERICANO (23NO REPORTADO2),URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76130000.0,1991.0,62-546,mtfdmb1912@unidatos.edu.co
12344,2021-12-30 09:38:00,VALLE,EL CERRITO,Miércoles,LOS SAMANES,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,BICICLETA,28.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76248000.0,1987.0,52-938,tnlpkc3117@unidatos.edu.co
12345,2021-12-30 19:35:00,VALLE,GUADALAJARA DE BUGA,Miércoles,LA MERCED,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,VEHICULO,PASAJERO MOTOCICLETA,31.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SUPERIOR,COLOMBIA,76111000.0,1984.0,22-336,kqjbkh3438@unidatos.edu.co
12346,2021-12-30 06:45:00,VALLE,TULUÁ,Miércoles,Urbanizacion Buenos Aires,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76834000.0,1991.0,50-889,correo650@unidatos.edu.co


In [60]:
days_aut["DIA"].value_counts()

Domingo      2765
Sábado       1848
Lunes        1679
Jueves       1637
Viernes      1635
Martes       1453
Miércoles    1378
Name: DIA, dtype: int64

## 6. Recalculation based on a different column

In [61]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
2094,1989.0,26.0
6455,1988.0,27.0
8135,1982.0,33.0
1862,1979.0,36.0
10074,1984.0,31.0
2825,1983.0,32.0
3859,1976.0,39.0
314,1990.0,25.0
11855,1972.0,43.0
3286,1960.0,55.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [62]:
import datetime
Edad_C = homicides_df
Edad_C['AÑO DE NACIMIENTO_CORREGIDO'] = datetime.datetime.now().year - Edad_C['EDAD']
Edad_C[['AÑO DE NACIMIENTO_CORREGIDO', "AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO_CORREGIDO,AÑO DE NACIMIENTO,EDAD
2427,1976.0,1968.0,47.0
6761,1958.0,1950.0,65.0
8814,2007.0,1999.0,16.0
4738,1954.0,1946.0,69.0
4486,1994.0,1986.0,29.0
10385,2004.0,1996.0,19.0
12277,1948.0,1940.0,75.0
9109,1978.0,1970.0,45.0
12202,2001.0,1993.0,22.0
3396,2000.0,1992.0,23.0


In [63]:
Edad_C['AÑO DE NACIMIENTO'] = Edad_C['AÑO DE NACIMIENTO_CORREGIDO']
Edad_C[['AÑO DE NACIMIENTO_CORREGIDO', "AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO_CORREGIDO,AÑO DE NACIMIENTO,EDAD
11709,1967.0,1967.0,56.0
10137,1983.0,1983.0,40.0
11129,1986.0,1986.0,37.0
259,1984.0,1984.0,39.0
2262,1994.0,1994.0,29.0
9878,2004.0,2004.0,19.0
10907,1967.0,1967.0,56.0
11992,1949.0,1949.0,74.0
12282,2003.0,2003.0,20.0
997,1989.0,1989.0,34.0


## 7. Conclusion

<span style="color:red">Make a summary of the different data quality problems found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating that specific problem.</span>

Consistencia (Cédula)
Problema: Malformación de algunos datos en el atributo "Cédula".
Estrategia: Utilizar funciones replace() para ajustar y estandarizar los valores de la "Cédula" y lograr consistencia.

Conformidad (Departamentos)
Problema: Varianza en los nombres de los departamentos.
Estrategia: Utilizar funciones replace() para ajustar los valores de departamentos que no están dentro del diccionario y lograr conformidad.

Duplicidad (Cédula)
Problema: Duplicidad de registros en la columna "Cédula".
Estrategia: Utilizar la función drop_duplicates() para eliminar registros duplicados y garantizar la conformidad de los datos.

Conformidad (Días de la semana)
Problema: Errores de digitación en los días de la semana.
Estrategia: Utilizar una función con un bucle for y la librería pylev.levenshtein para corregir los datos y lograr conformidad.

Precisión (Año de Nacimiento)
Problema: Errores en el año de nacimiento.
Estrategia: Calcular el año de nacimiento correcto restando la "Edad" al año actual para mejorar la precisión de los datos.